In [6]:
import numpy as np
import logging
import pickle

logging.basicConfig(filename=f'queue_state.log', filemode='w', format='%(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [7]:
def conditions_generation(l: int, mu: int, size: int) -> tuple[np.ndarray, np.ndarray]:
    return [[np.random.exponential(scale=1/l, size=1)[0], np.random.exponential(1/mu, size=1)[0]] for _ in range(size)]

l = 2
mu = 1
size =100
'T1 - время между приходами i-1 и i требований'
'T2 - время обсуживания i-ого требования'

conditions = conditions_generation(l=l, mu=mu, size=size)
conditions[0][0] = 0

In [8]:
class Model():
    
    def __init__(self, max_queue_capacity, conditions):
        
        self.max_queue_capacity = max_queue_capacity
        self.conditions = conditions
        self.time_log = []
        self.queue_log = []
        self.server_log = []
        
    def run(self):
        current_time = 0
        remaining_server_time = 0
        current_queue_occupancy = []        
        
        while len(self.conditions) != 0:
            
            t1, t2 = self.conditions.pop(0)
            logger.info(f"Current itteration t1: {t1} - t2: {t2}")
            logger.info(f"Before itter. Server state: {remaining_server_time}, Queue state: {current_queue_occupancy}")
            
            if t1 >= sum(current_queue_occupancy) + remaining_server_time:
                t1 -= remaining_server_time
                
                current_time += remaining_server_time           
                
                while len(current_queue_occupancy) != 0 and t1 > 0:
                    remaining_server_time = current_queue_occupancy.pop(0)
                    t1 -= remaining_server_time
                    
                    current_time += remaining_server_time
                    self.dump(current_time, current_queue_occupancy, remaining_server_time)

                if t1 > 0:
                    current_time += t1
                
                remaining_server_time = t2

            else:
                
                if remaining_server_time > t1:
                    remaining_server_time -= t1
                    current_time += t1
                    
                    if len(current_queue_occupancy) == self.max_queue_capacity:
                        logger.info(f"Queue is full. Drop this operations.\n")
                    else:
                        current_queue_occupancy.append(t2)

                    self.dump(current_time, current_queue_occupancy, remaining_server_time)

                else:
                    
                    t1 -= remaining_server_time
                    remaining_server_time = 0
                    current_time += remaining_server_time
                    
                    while len(current_queue_occupancy) != 0 and t1 != 0:
                        
                        remaining_server_time = current_queue_occupancy.pop(0) 
                        
                        self.dump(current_time, current_queue_occupancy, remaining_server_time)
                        
                        if t1 - remaining_server_time > 0:
                            
                            t1 -= remaining_server_time
                            current_time += remaining_server_time

                        else:
                            
                            remaining_server_time -= t1
                            current_time += remaining_server_time                    
                            t1 = 0
                                                    
                    current_queue_occupancy.append(t2)
                    
                    self.dump(current_time, current_queue_occupancy, remaining_server_time)
                    
            logger.info(f"After itter. Server state: {remaining_server_time}, Queue state: {current_queue_occupancy}\n")
        
        
    def dump(self, current_time, current_queue_occupancy, remaining_server_time):
        
        self.time_log.append(current_time)
        self.queue_log.append(current_queue_occupancy)
        self.server_log.append(remaining_server_time) 
        
    def save(self):
        
        with open('time.pkl', 'wb') as tf:
            pickle.dump(self.time_log, tf)
        with open('queue.pkl', 'wb') as qf:
            pickle.dump(self.queue_log, qf)    
        with open('server.pkl', 'wb') as sf:
            pickle.dump(self.server_log, sf)

In [9]:
queue_capacity = 20
conditions = conditions_generation(l=l, mu=mu, size=size)
conditions[0][0] = 0

model = Model(max_queue_capacity=queue_capacity, conditions=conditions)

In [10]:
model.run()

In [12]:
model.save()